In [1]:
import pandas as pd
import os
from google.cloud import bigquery
import json
from google.oauth2 import service_account
import db_dtypes 

In [2]:
# Obtain the key from the environment variable
service_account_key =os.getenv('GCP_ACCESS_KEY')
if not service_account_key:
    raise ValueError("The GCP_SERVICE_ACCOUNT_KEY environment variable is not set")

key_data = json.loads(service_account_key)

credentials = service_account.Credentials.from_service_account_info(
    key_data,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


In [3]:
service_acc_key=os.getenv('GCP_ACCESS_KEY')

In [4]:
project_id = os.getenv('GOOGLE_PROJECT_MAIN_FP')

In [5]:
client = bigquery.Client(credentials=credentials,project=project_id)

In [6]:
sql = """
SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 10
 """

In [11]:
# import pandas_gbq
# df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials, progress_bar_type=None)

In [7]:

df = client.query(sql).to_dataframe()

In [8]:
df.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


In [10]:
df.describe()

,trip_id,duration_sec,start_station_id,end_station_id,bike_number
count,10.0,10.0,10.0,10.0,10.0
mean,721404.7,19944.1,2.0,2.0,251.5
std,428466.557797,37212.663927,0.0,0.0,229.934894
min,142982.0,586.0,2.0,2.0,109.0
25%,336168.5,1990.0,2.0,2.0,124.75
50%,766215.0,4580.0,2.0,2.0,149.0
75%,1131537.0,7248.75,2.0,2.0,186.0
max,1245113.0,115572.0,2.0,2.0,702.0


In [11]:
my_sql = """
SELECT DATE(start_date) AS start_date, start_station_id, count(*) AS Trip_Count FROM bigquery-public-data.san_francisco.bikeshare_trips group by 1,2
 """

dataset = client.query(my_sql).to_dataframe()

In [12]:
dataset.head()

,start_date,start_station_id,Trip_Count
0,2016-06-23,65,48
1,2013-09-06,70,47
2,2016-06-02,50,70
3,2016-04-18,74,61
4,2015-09-02,74,67


In [13]:
dataset['start_station_id'].nunique()

74

#### BigQuery Model Creations

In [14]:
my_sql = """


CREATE OR REPLACE MODEL
  xx.test_table.bq_ml_covid_model OPTIONS ( MODEL_TYPE = 'ARIMA_PLUS',
    TIME_SERIES_TIMESTAMP_COL='DATE_ST',
    TIME_SERIES_DATA_COL='NO_TRIP',
    TIME_SERIES_ID_COL='start_station_id' ) AS
SELECT
  *
FROM (
  SELECT
    DATE(start_date) AS DATE_ST,
    start_station_id,
    COUNT(*) AS NO_TRIP
  FROM
    bigquery-public-data.san_francisco.bikeshare_trips
  GROUP BY
    1,
    2)

 """

model_creation = client.query(my_sql).to_dataframe()

#### Model Evaluations

In [14]:
eval_sql = """


SELECT * FROM

ML.ARIMA_EVALUATE ( 
  MODEL `friendly-plane-294914.test_table.bq_ml_covid_model`
)
 """
model_eval = client.query(eval_sql).to_dataframe()

In [15]:
model_eval.head()

,start_station_id,non_seasonal_p,non_seasonal_d,non_seasonal_q,has_drift,log_likelihood,AIC,variance,seasonal_periods,has_holiday_effect,has_spikes_and_dips,has_step_changes,error_message
0,2,0,1,5,False,-2522.007743,5056.015487,5.744745,"[WEEKLY, YEARLY]",False,False,False,
1,3,0,1,5,True,-1863.248955,3740.497909,1.733974,[NO_SEASONALITY],False,True,True,
2,4,0,1,5,False,-2011.124670,4034.249340,2.256120,[WEEKLY],False,False,False,
3,5,0,1,5,True,-1444.365936,2902.731872,0.800527,[WEEKLY],False,True,True,
4,6,1,1,2,True,-1908.711024,3827.422049,1.883347,"[WEEKLY, YEARLY]",False,True,True,


#### ARIMA Evaluation explanation

##### start_station_id: 
This is likely an identifier for a specific account or dataset within Google BigQuery. It helps in distinguishing between different models or datasets.

##### non_seasonal_p, non_seasonal_d, non_seasonal_q: These are the standard ARIMA model parameters.

##### non_seasonal_p (Autoregressive part): 
Indicates the number of lag observations in the model (lag order). Here, it's 1, meaning the model uses 1 previous time point to predict the current value.
##### non_seasonal_d (Integrated part): 
The number of differences used to make the time series stationary. Here, it's 1, meaning the data is differenced once.
non_seasonal_q (Moving Average part): The size of the moving average window. Here, it's 1, suggesting the model uses the error term from the previous time point.

##### has_drift: 
This indicates whether the model includes a drift term or not. A drift term in a time series model allows for a linear trend over time. True here means the model accounts for some kind of linear trend in the data.

##### log_likelihood: 
This is a measure of how well the model fits the data. It's the logarithm of the likelihood function, which measures the probability of observing the data given the model parameters. Higher values typically indicate a better model fit.

##### AIC (Akaike Information Criterion):
 A widely used criterion for model selection. It considers the goodness of fit of the model and the number of parameters used. Lower AIC values generally indicate a better model, as they suggest a model is both simple and fits the data well.

##### variance: 
This likely refers to the variance of the residuals (errors) of the model. It's a measure of the dispersion of these errors. In time series forecasting, smaller variance is typically better, indicating that the model's predictions are more consistently close to the actual values.

##### seasonal_periods: 
Indicates the type of seasonality in the data. [YEARLY] suggests the model has identified an annual seasonal pattern in the data.

##### has_holiday_effect: 
Indicates whether the model accounts for holiday effects. False means the model does not consider holidays as a special factor affecting the time series.

##### has_spikes_and_dips:
This suggests the model accounts for sudden increases (spikes) and decreases (dips) in the time series data. True means the model considers these anomalies.

##### has_step_changes: 
Indicates whether the model accounts for step changes or level shifts in the time series. False means the model does not account for sudden, lasting changes in the level of the series.

#### Anomaly Detection

In [13]:
eval_sql = """


SELECT
  *
FROM
  ML.DETECT_ANOMALIES ( 
    MODEL `xxx.test_table.bq_ml_covid_model`,
    STRUCT(0.8 AS anomaly_prob_threshold)  )

 """
model_eval = client.query(eval_sql).to_dataframe()

In [14]:
model_eval.head()

,start_station_id,DATE_ST,NO_TRIP,is_anomaly,lower_bound,upper_bound,anomaly_probability
0,2,2013-08-29 00:00:00+00:00,5.0,False,1.926590,8.073410,0.000000
1,2,2013-08-30 00:00:00+00:00,4.0,False,0.582555,6.729375,0.113714
2,2,2013-08-31 00:00:00+00:00,7.0,True,0.381124,6.527944,0.861040
3,2,2013-09-01 00:00:00+00:00,6.0,True,-4.701196,1.445624,0.998128
4,2,2013-09-02 00:00:00+00:00,6.0,False,4.116288,10.263108,0.380829


In [15]:
model_eval.shape

(73059, 7)

In [16]:
model_eval['start_station_id'].value_counts()

start_station_id
2     1099
62    1099
46    1099
3     1099
48    1099
      ... 
21     333
89      87
90      28
91      27
88      15
Name: count, Length: 74, dtype: Int64

In [18]:
query_papams = [
    bigquery.ScalarQueryParameter("subscriber_type", "STRING", "Customer"),
]
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_papams

sql = """ SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
WHERE subscriber_type = @subscriber_type
  """

query_job = client.query(sql, job_config=job_config)
query_job.result()
df_result = query_job.to_dataframe()
print(" query results loaded to the table")

 query results loaded to the table


In [19]:
df_result

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer
...,...,...,...,...,...,...,...,...,...,...,...
136804,1323441,102,2016-08-19 16:56:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-19 16:58:00+00:00,Cyril Magnin St at Ellis St,91,473,55104,Customer
136805,1327216,7913,2016-08-23 14:02:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-23 16:14:00+00:00,Cyril Magnin St at Ellis St,91,377,90049,Customer
136806,1320788,564,2016-08-18 06:41:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-18 06:50:00+00:00,Cyril Magnin St at Ellis St,91,295,20025,Customer
136807,1320548,7000,2016-08-17 18:26:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-17 20:22:00+00:00,Cyril Magnin St at Ellis St,91,226,nil,Customer


In [ ]:
sql = """
SELECT * FROM `bigquery-public-data.covid19_geotab_mobility_impact.city_congestion` 
 """
model_creation = client.query(sql).to_dataframe()